## 네이버 영화 감정 분석

In [1]:
BASIC_PATH = "/dataset"

### [1] 라이브러리 설정

In [2]:
import numpy as np
import pandas as pd
import os
import re
import json

from konlpy.tag import Okt

from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

### [2] Preprocessing 과정 진행

In [3]:
def preprocessing(review, okt, remove_stopwords = False, stop_words = []):
    
    # 1. 한글 및 공백을 제외한 모든 문자 제거 -> 한글만 남기겠다.
    review_text = re.sub("[^가-힣ㄱ-ㅎㅏ - | \\s]", "", review)
    
    # 2. Okt 객체를 활용해 형태소 단위로 문장을 나눈다(tokenzing)
    word_review = okt.morphs(review_text, stem=True)
    
    # 3. 불용어 제거하기
    if remove_stopwords:
        word_review = [token for token in word_review if not token in stop_words]
        
    clean_review = " ".join(word_review)
    
    return clean_review

#### [2-1] 데이터 불러오기

In [4]:
TOTAL_RATINGS = '/rating.txt'
TRAIN_RATINGS = '/ratings_train.txt'
TEST_RATINGS = '/ratings_test.txt'

In [5]:
train_data = pd.read_csv(
    BASIC_PATH + TRAIN_RATINGS, header = 0, delimiter = '\t', quoting=3
)

In [6]:
train_data.head(5)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [7]:
train_data = train_data.dropna(axis=0) # 결측치가 있는 행 모두 삭제

#### [2-2] stop_word 지정 및 데이터 전처리 진행

In [8]:
stop_words = ['은', '는', '이', '가', '하', '아', '들', '것', '의', '있', '되', '수', '보', '주', '등', '한']

okt = Okt() #형태소 분석기 선언
clean_train_review = []

for review in train_data['document']:
    clean_train_review.append(preprocessing(review, okt, remove_stopwords = True, stop_words = stop_words))

In [9]:
clean_train_review[10]

'걍 인피니트 짱 이다 진짜 짱 이다'

#### [2-3] 전처리를 통해 ''가 된 데이터들은 제거

In [10]:
print(len(train_data))

149995


In [11]:
train_label = train_data['label'].tolist()
new_clean_train_review, new_train_label = [], []
for idx, clean in enumerate(clean_train_review):
    if clean != "": 
        new_clean_train_review.append(clean)
        new_train_label.append(train_label[idx])

In [12]:
print(len(new_clean_train_review))
print(len(new_train_label))

148653
148653


#### [2-4] test dataset도 똑같이 진행

In [13]:
test_data = pd.read_csv(
    BASIC_PATH + TEST_RATINGS, header = 0, delimiter = '\t', quoting=3
)

In [14]:
test_data = test_data.dropna(axis=0) # 결측치가 있는 행 모두 삭제

In [15]:
# data preprocessing
clean_test_reviews = []
for review in test_data["document"]:
    clean_test_reviews.append(preprocessing(review, okt, remove_stopwords = True, stop_words = stop_words))

In [18]:
print(len(test_data))

new_clean_test_review = []
for idx, clean in enumerate(clean_test_reviews):
    if clean != "": 
        new_clean_test_review.append(clean)

49997


In [19]:
print(len(new_clean_test_review))

49539


### [3] clean data 새 프레임 생성

In [20]:
# 데이터 전처리 과정 진행한 데이터를 다시 새 프레임으로 생성
clean_train_df = pd.DataFrame({'document' : new_clean_train_review, 'label' : new_train_label})

In [21]:
# 데이터 전처리 과정 진행한 데이터를 다시 새 프레임으로 생성
clean_test_df = pd.DataFrame({'document' : new_clean_test_review})

### [4] 데이터 전처리 끝난 데이터 세트 파일로 저장

In [22]:
TRAIN_CLEAN_DATA = BASIC_PATH + "/train_clean.csv"

In [23]:
TEST_DATA = BASIC_PATH + "/test_clean.csv"

In [24]:
clean_train_df.to_csv(TRAIN_CLEAN_DATA, index = False)
clean_test_df.to_csv(TEST_DATA, index = False)